# Imports

In [127]:
import pandas as pd
import os
import seaborn as sns
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from scipy.stats import mode
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS

import numpy as np

# Loading the Dataframe

In [2]:
pwd

'/home/rafael/code/rafabertolace/OnThePitch/notebooks'

## Merging the Seasons csv files (2019-2020 untill 2021-2022)

In [6]:
print(os.listdir('../raw_data/'))

['france', 'germany', 'eredivisie', 'turkey', 'extract-betsentiment-com.csv', 'england', 'greece', 'italy', 'spain', 'scotland', 'portugal', 'belgium']


In [11]:
files = [file for file in os.listdir('../raw_data/all/') if file.endswith('.csv')]
data = pd.DataFrame()

for file in files:
    df = pd.read_csv('../raw_data/all/' + file)
    df['country']=str(file)[0:2]
    data = pd.concat([data, df])

In [14]:
data

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,country,Unnamed: 105
0,SC0,01/08/2020,12:30,Aberdeen,Rangers,0,1,A,0.0,1.0,...,1.83,2.02,1.90,2.00,1.94,2.12,1.85,1.99,sc,NaN
1,SC0,01/08/2020,15:00,Dundee United,St Johnstone,1,1,D,1.0,0.0,...,1.93,1.93,1.91,1.99,2.00,1.99,1.92,1.93,sc,NaN
2,SC0,01/08/2020,15:00,Hibernian,Kilmarnock,2,1,H,2.0,1.0,...,1.95,1.90,1.96,1.94,2.02,1.94,1.95,1.88,sc,NaN
3,SC0,01/08/2020,15:00,St Mirren,Livingston,1,0,H,1.0,0.0,...,1.85,2.00,1.88,2.01,1.90,2.08,1.83,2.01,sc,NaN
4,SC0,02/08/2020,16:30,Celtic,Hamilton,5,1,H,2.0,1.0,...,1.98,1.88,2.01,1.88,2.04,1.92,1.96,1.87,sc,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,G1,18/07/2020,17:15,Larisa,Xanthi,0,0,D,0.0,0.0,...,2.05,1.80,2.07,1.78,2.14,1.83,2.06,1.78,Gr,NaN
236,G1,18/07/2020,17:15,Panetolikos,Volos NFC,1,0,H,1.0,0.0,...,1.83,2.02,1.85,2.00,1.88,2.07,1.82,2.01,Gr,NaN
237,G1,19/07/2020,18:00,Olympiakos,AEK,3,0,H,2.0,0.0,...,1.95,1.90,1.93,1.93,2.00,1.97,1.92,1.91,Gr,NaN
238,G1,19/07/2020,18:00,Panathinaikos,OFI Crete,3,2,H,3.0,2.0,...,1.93,1.93,1.90,1.95,1.99,2.00,1.90,1.92,Gr,NaN


In [15]:
for col_name in data.columns: 
    print(col_name)

Div
Date
Time
HomeTeam
AwayTeam
FTHG
FTAG
FTR
HTHG
HTAG
HTR
Referee
HS
AS
HST
AST
HF
AF
HC
AC
HY
AY
HR
AR
B365H
B365D
B365A
BWH
BWD
BWA
IWH
IWD
IWA
PSH
PSD
PSA
WHH
WHD
WHA
VCH
VCD
VCA
MaxH
MaxD
MaxA
AvgH
AvgD
AvgA
B365>2.5
B365<2.5
P>2.5
P<2.5
Max>2.5
Max<2.5
Avg>2.5
Avg<2.5
AHh
B365AHH
B365AHA
PAHH
PAHA
MaxAHH
MaxAHA
AvgAHH
AvgAHA
B365CH
B365CD
B365CA
BWCH
BWCD
BWCA
IWCH
IWCD
IWCA
PSCH
PSCD
PSCA
WHCH
WHCD
WHCA
VCCH
VCCD
VCCA
MaxCH
MaxCD
MaxCA
AvgCH
AvgCD
AvgCA
B365C>2.5
B365C<2.5
PC>2.5
PC<2.5
MaxC>2.5
MaxC<2.5
AvgC>2.5
AvgC<2.5
AHCh
B365CAHH
B365CAHA
PCAHH
PCAHA
MaxCAHH
MaxCAHA
AvgCAHH
AvgCAHA
country
Unnamed: 105


# Features Engineering

In [16]:
def feature_engineering(data, b=20, binned=False):
    '''
    This function creates all the columns that will be needed to create the analysis 
    and return the dataframe with all this changes
    
    b is the number of bins that we want to work with. Our start value for b will be 20.
        '''
    #------------------------Number of Goals, Over and Under -----------------------------------
    
    # total number of goals = goals from the home team + goals from visiting team
    data['nb_goals']=data['FTHG']+data['FTAG']

    # boolean: true or false regarding whether they were more than 2.5 goals
    data['over_2.5_goals']=data['nb_goals']>2.5

    # boolean: true or false regarding whether they were less than 2.5 goals
    data['under_2.5_goals']=data['nb_goals']<2.5
    
    #-----------------------------Payout Opening ----------------------------------------------
    
    # payout under 2.5 for Average OPENING odds
    data['payout_avg_under_2.5'] = data['under_2.5_goals']*data['Avg<2.5']

    # payout over 2.5 for Average OPENING odds
    data['payout_avg_over_2.5'] = data['over_2.5_goals']*data['Avg>2.5']

    #payout UNDER 2.5 for PINACLE specifically
    data['payout_under_2.5_pinacle'] = data['under_2.5_goals']*data['P<2.5']

    #payout OVER 2.5 for PINACLE specifically
    data['payout_over_2.5_pinacle'] = data['over_2.5_goals']*data['P>2.5']

    #payout UNDER 2.5 for 365 specifically
    data['payout_under_2.5_365'] = data['under_2.5_goals']*data['B365<2.5']

    #payout OVER 2.5 for 365 specifically
    data['payout_over_2.5_365'] = data['over_2.5_goals']*data['B365>2.5']
    
    #------------------------------Payout Closing --------------------------------------------
    
    # payout under 2.5 for Average CLOSING odds
    data['payout_avg_under_closing_2.5'] = data['under_2.5_goals']*data['AvgC<2.5']

    # payout over 2.5 for Average CLOSING odds
    data['payout_avg_over_closing_2.5'] = data['over_2.5_goals']*data['AvgC>2.5']

    #payout UNDER 2.5 for PINACLE closing ddds specifically
    data['payout_under_2.5_pinacle_closing'] = data['under_2.5_goals']*data['PC<2.5']

    #payout OVER 2.5 for PINACLE closing odds specifically
    data['payout_over_2.5_pinacle_closing'] = data['over_2.5_goals']*data['PC>2.5']

    #payout UNDER 2.5 for 365 closing odds specifically
    data['payout_under_2.5_365_closing'] = data['under_2.5_goals']*data['B365C<2.5']

    #payout OVER 2.5 for 365 closing odds specifically
    data['payout_over_2.5_365_closing'] = data['over_2.5_goals']*data['B365C>2.5']
    
    #-------------------------- Implied Probability Opening ----------------------------------------
    
    #Implied Probability UNDER 2.5 goals for for overall market opening odds (Avg) 
    data['Implied Probability <2.5 avg']=1/data['Avg<2.5']*100

    #Implied Probability OVER 2.5 goals for for overall market opening odds (Avg) 
    data['Implied Probability >2.5 avg']=1/data['Avg>2.5']*100

    #Implied Probability UNDER 2.5 goals for PINACLE
    data['Implied Probability <2.5 pinacle']=1/data['P<2.5']*100

    #Implied Probability OVER 2.5 goals for PINACLE
    data['Implied Probability >2.5 pinacle']=1/data['P>2.5']*100

    #Implied Probability UNDER 2.5 goals for 365
    data['Implied Probability <2.5 365']=1/data['B365<2.5']*100

    #Implied Probability OVER 2.5 goals for 365
    data['Implied Probability >2.5 365']=1/data['B365>2.5']*100
    
    #------------------------- Implied Probability Closing -----------------------------------
    
    #Implied Probability UNDER 2.5 goals for overall market closing odds (AvgC)
    data['Implied Probability <2.5 avg closing']=1/data['AvgC<2.5']*100

    #Implied Probability OVER 2.5 goals for overall market closing odds (AvgC)
    data['Implied Probability >2.5 avg closing']=1/data['AvgC>2.5']*100

    #Implied Probability UNDER 2.5 goals for PINACLE closing odds
    data['Implied Probability <2.5 pinacle closing']=1/data['PC<2.5']*100

    #Implied Probability OVER 2.5 goals for PINACLE closing odds
    data['Implied Probability >2.5 pinacle closing']=1/data['PC>2.5']*100

    #Implied Probability UNDER 2.5 goals for 365 closing odds
    data['Implied Probability <2.5 365 closing']=1/data['B365C<2.5']*100

    #Implied Probability OVER 2.5 goals for 365 closing odds
    data['Implied Probability >2.5 365 closing']=1/data['B365C>2.5']*100
    
    #---------------------------- Binning IP Opening -------------------------------------

    b=b
    bins = np.arange(0, 101, int(100/b))
    bins = bins.tolist()

    #Binning UNDER 2.5 Average Market opening odds
    data['binned <2.5 avg'] = pd.cut(data['Implied Probability <2.5 avg'], bins)

    #Binning Over 2.5 Average Market opening odds
    data['binned >2.5 avg'] = pd.cut(data['Implied Probability >2.5 avg'], bins)

    #Binned UNDER 2.5 Pinnacle opening odds
    data['binned <2.5 pinacle'] = pd.cut(data['Implied Probability <2.5 pinacle'], bins)

    #Binned OVER 2.5 Pinnacle
    data['binned >2.5 pinacle'] = pd.cut(data['Implied Probability >2.5 pinacle'], bins)

    #Binned UNDER 2.5 bet365 OPENING odds
    data['binned <2.5 365'] = pd.cut(data['Implied Probability <2.5 365'], bins)

    #Binned OVER 2.5 bet365 OPENING odds
    data['binned >2.5 365'] = pd.cut(data['Implied Probability >2.5 365'], bins)
    
    #----------------------------- Binning IP Closing ------------------------------------------------

    #Binning UNDER 2.5 Average Market closing odds
    data['binned <2.5 avg closing'] = pd.cut(data['Implied Probability <2.5 avg closing'], bins)

    #Binning OVER 2.5 Average Market closing odds
    data['binned >2.5 avg closing'] = pd.cut(data['Implied Probability >2.5 avg closing'], bins)

    #Binned UNDER 2.5 Pinnacle closing odds
    data['binned <2.5 pinacle closing'] = pd.cut(data['Implied Probability <2.5 pinacle closing'], bins)

    #Binned OVER 2.5 Pinnacle CLOSING odds
    data['binned >2.5 pinacle closing'] = pd.cut(data['Implied Probability >2.5 pinacle closing'], bins)

    #Binned UNDER 2.5 bet365 CLOSING odds
    data['binned <2.5 365 closing'] = pd.cut(data['Implied Probability <2.5 365 closing'], bins)

    #Binned OVER 2.5 bet365 CLOSING odds
    data['binned >2.5 365 closing'] = pd.cut(data['Implied Probability >2.5 365 closing'], bins)
    
    #---------------------------- Binning Odds Opening ----------------------------------------------------
    
    bins2 = [1, 1.5, 2, 3, 99999]

    #Binning UNDER 2.5 Average Market opening odds
    data['binned odds <2.5 avg'] = pd.cut(data['Avg<2.5'], bins2)

    #Binning Over 2.5 Average Market opening odds
    data['binned odds >2.5 avg'] = pd.cut(data['Avg>2.5'], bins2)

    #Binned UNDER 2.5 Pinnacle opening odds
    data['binned odds <2.5 pinacle'] = pd.cut(data['P<2.5'], bins2)

    #Binned OVER 2.5 Pinnacle
    data['binned odds >2.5 pinacle'] = pd.cut(data['P>2.5'], bins2)

    #Binned UNDER 2.5 bet365 OPENING odds
    data['binned odds <2.5 365'] = pd.cut(data['B365<2.5'], bins2)

    #Binned OVER 2.5 bet365 OPENING odds
    data['binned odds >2.5 365'] = pd.cut(data['B365>2.5'], bins2)
    
    #----------------------------- Binning Odds Closing ----------------------------------------------------------
    
    #Binning UNDER 2.5 Average Market opening odds
    data['binned odds <2.5 avg closing'] = pd.cut(data['AvgC<2.5'], bins2)

    #Binning Over 2.5 Average Market opening odds
    data['binned odds >2.5 avg closing'] = pd.cut(data['AvgC>2.5'], bins2)

    #Binned UNDER 2.5 Pinnacle opening odds
    data['binned odds <2.5 pinacle closing'] = pd.cut(data['PC<2.5'], bins2)

    #Binned OVER 2.5 Pinnacle
    data['binned odds >2.5 pinacle closing'] = pd.cut(data['PC>2.5'], bins2)

    #Binned UNDER 2.5 bet365 OPENING odds
    data['binned odds <2.5 365 closing'] = pd.cut(data['B365C<2.5'], bins2)

    #Binned OVER 2.5 bet365 OPENING odds
    data['binned odds >2.5 365 closing'] = pd.cut(data['B365C>2.5'], bins2)
    
    
    #----------------------------- Other Features from D3 ------------------------------------------------------
    
    data['Pin_pays_better_under_boolean'] = data['PC<2.5'] > data['AvgC<2.5']
    data['Pin_pays_better_under_difference'] = data['PC<2.5'] / data['AvgC<2.5']
    data['%vig_p'] = (1 - (1 / (1/data['PC>2.5'] + 1/data['PC<2.5'])))*100
    data['%vig_avg'] = (1 - (1 / (1/data['AvgC>2.5'] + 1/data['AvgC<2.5'])))*100
    data['PC<2.5_P_boolean'] = data['PC<2.5'] < data['P<2.5']
    data['PC<2.5_P_relative_diff'] = data['PC<2.5'] / data['P<2.5']
    
    #----------------------- Odds and probability of the home team scoring under 2.5 -------------------------------
    
    lst1 = []
    lst2 = []
    for i, team in enumerate(data['HomeTeam']):
        date = data['Date'].iloc[i]
        total = len(data[(data['HomeTeam'] == team) & (data['Date'] < date)])
        n_under_home = data[(data['HomeTeam'] == team) & (data['Date'] < date)]['under_2.5_goals'].value_counts()
        try:
            lst1.append(1/(n_under_home[1]/total))
            lst2.append(n_under_home[1]/total)
        except:
            lst1.append(np.nan)
            lst2.append(np.nan)

    data['odds_home_under'] = lst1
    data['prob_home_under'] = lst2
    
    #binning the probability of the home team to have a game of less than 2.5 score
    data['binned prob_home_under'] = pd.cut(data['prob_home_under']*100, bins)
    
    
    #----------------------- Odds and probability of the away team scoring under 2.5 -------------------------------
    
    lst3 = []
    lst4  = []
    for i, team in enumerate(data['AwayTeam']):
        date = data['Date'].iloc[i]
        total2 = len(data[(data['AwayTeam'] == team) & (data['Date'] < date)])
        n_under_away2 = data[(data['AwayTeam'] == team) & (data['Date'] < date)]['under_2.5_goals'].value_counts()
        try:
            lst3.append(1/(n_under_away2[1] / total2))
            lst4.append(n_under_away2[1] / total2)
        except:
            lst3.append(np.nan)
            lst4.append(np.nan)

    data['odds_away_under'] = lst3
    data['prob_away_under'] = lst4
    
    #binning the probability of the away team to have a game of less than 2.5 score
    data['binned prob_away_under'] = pd.cut(data['prob_away_under']*100, bins)

    #-------------------------- Creating the prob and odds of the game -----------------------------------------------
    '''the mean between the probability of the home team to have a score of under 2.5 and the probability 
    of the away team to do the same'''
    
    data['odds_game'] = (data['odds_away_under'] +  data['odds_home_under']) / 2
    data['prob_game'] = (data['prob_away_under'] + data['prob_home_under']) / 2
    
    #-------------------------- OneHotEncoding the binned probabilities columns ------------------------------------------
    

    if b == 5:
        data = data[~data['binned prob_home_under'].isna()]
        ohe = OneHotEncoder(sparse=False)
        ohe.fit(data[['binned prob_home_under']])
        bins_encoded = ohe.transform(data[['binned prob_home_under']])
        data["0, 20"], data["20, 40"], data["40, 60"], data["60, 80"], data["80, 100"] = bins_encoded.T
        
    if b == 10:
        data = data[~data['binned prob_home_under'].isna()]
        ohe = OneHotEncoder(sparse=False)
        ohe.fit(data[['binned prob_home_under']])
        bins_encoded = ohe.transform(data[['binned prob_home_under']])
        data["0, 10"], data["10, 20"], data["20, 30"], data["30, 40"], data["40, 50"], data["50, 60"], \
        data["60, 70"], data["70, 80"], data["80, 90"], data["90, 100"] = bins_encoded.T
        
    if b == 20:
        data = data[~data['binned prob_home_under'].isna()]
        ohe = OneHotEncoder(sparse=False)
        ohe.fit(data[['binned prob_home_under']])
        bins_encoded = ohe.transform(data[['binned prob_home_under']])
        data["0, 5"], data["5, 10"], data["10, 15"], data["15, 20"], data["20, 25"], data["25, 30"], \
        data["30, 35"], data["35, 40"], data["40, 45"], data["45, 50"], data["50, 55"], data["55, 60"], \
        data["60, 65"], data["65, 70"], data["70, 75"], data["75, 80"], data["80, 85"], data["85, 90"], \
        data["90, 95"], data["95, 100"]= bins_encoded.T
    
    
    #------------------------------------ Cleaning the data ---------------------------------------------------------
    
    #data = data.dropna(subset=['HomeTeam', 'AwayTeam'], how='any')
    data = data[~data['HomeTeam'].isna()]
    data = data[~data['AwayTeam'].isna()]
    data = data[~data['PC>2.5'].isna()]
    data.drop(columns=['Referee','Unnamed: 105'], inplace=True) #, 'Unnamed: 105' 'Referee', 
    #data.dropna()
    
    return data

In [17]:
data = feature_engineering(data, b=5)

/tmp/ipykernel_2640/4191015178.py:253: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["0, 20"], data["20, 40"], data["40, 60"], data["60, 80"], data["80, 100"] = bins_encoded.T
/tmp/ipykernel_2640/4191015178.py:253: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["0, 20"], data["20, 40"], data["40, 60"], data["60, 80"], data["80, 100"] = bins_encoded.T
/tmp/ipykernel_2640/4191015178.py:253: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

### Number of Goals, Over and Under

In [18]:
# total number of goals = goals from the home team + goals from visiting team
data['nb_goals']=data['FTHG']+data['FTAG']

# boolean: true or false regarding whether they were more than 2.5 goals
data['over_2.5_goals']=data['nb_goals']>2.5

# boolean: true or false regarding whether they were less than 2.5 goals
data['under_2.5_goals']=data['nb_goals']<2.5

### Payout

  Payout of betting on over/under 2.5 goals: we get 0 if we lose the bet, we get the Avg if we win the bet (Avg = market average of the odds)


#### Payout Opening

In [19]:
# payout under 2.5 for Average OPENING odds
data['payout_avg_under_2.5'] = data['under_2.5_goals']*data['Avg<2.5']

# payout over 2.5 for Average OPENING odds
data['payout_avg_over_2.5'] = data['over_2.5_goals']*data['Avg>2.5']

#payout UNDER 2.5 for PINACLE specifically
data['payout_under_2.5_pinacle'] = data['under_2.5_goals']*data['P<2.5']

#payout OVER 2.5 for PINACLE specifically
data['payout_over_2.5_pinacle'] = data['over_2.5_goals']*data['P>2.5']

#payout UNDER 2.5 for 365 specifically
data['payout_under_2.5_365'] = data['under_2.5_goals']*data['B365<2.5']

#payout OVER 2.5 for 365 specifically
data['payout_over_2.5_365'] = data['over_2.5_goals']*data['B365>2.5']

#### Payout Closing

In [20]:
# payout under 2.5 for Average CLOSING odds
data['payout_avg_under_closing_2.5'] = data['under_2.5_goals']*data['AvgC<2.5']

# payout over 2.5 for Average CLOSING odds
data['payout_avg_over_closing_2.5'] = data['over_2.5_goals']*data['AvgC>2.5']

#payout UNDER 2.5 for PINACLE closing ddds specifically
data['payout_under_2.5_pinacle_closing'] = data['under_2.5_goals']*data['PC<2.5']

#payout OVER 2.5 for PINACLE closing odds specifically
data['payout_over_2.5_pinacle_closing'] = data['over_2.5_goals']*data['PC>2.5']

#payout UNDER 2.5 for 365 closing odds specifically
data['payout_under_2.5_365_closing'] = data['under_2.5_goals']*data['B365C<2.5']

#payout OVER 2.5 for 365 closing odds specifically
data['payout_over_2.5_365_closing'] = data['over_2.5_goals']*data['B365C>2.5']

### Implied Probability

#### Implied Probability Opening

In [21]:
#Implied Probability UNDER 2.5 goals for for overall market opening odds (Avg) 
data['Implied Probability <2.5 avg']=1/data['Avg<2.5']*100

#Implied Probability OVER 2.5 goals for for overall market opening odds (Avg) 
data['Implied Probability >2.5 avg']=1/data['Avg>2.5']*100

#Implied Probability UNDER 2.5 goals for PINACLE
data['Implied Probability <2.5 pinacle']=1/data['P<2.5']*100

#Implied Probability OVER 2.5 goals for PINACLE
data['Implied Probability >2.5 pinacle']=1/data['P>2.5']*100

#Implied Probability UNDER 2.5 goals for 365
data['Implied Probability <2.5 365']=1/data['B365<2.5']*100

#Implied Probability OVER 2.5 goals for 365
data['Implied Probability >2.5 365']=1/data['B365>2.5']*100

#### Implied Probability Closing

In [22]:
#Implied Probability UNDER 2.5 goals for overall market closing odds (AvgC)
data['Implied Probability <2.5 avg closing']=1/data['AvgC<2.5']*100

#Implied Probability OVER 2.5 goals for overall market closing odds (AvgC)
data['Implied Probability >2.5 avg closing']=1/data['AvgC>2.5']*100

#Implied Probability UNDER 2.5 goals for PINACLE closing odds
data['Implied Probability <2.5 pinacle closing']=1/data['PC<2.5']*100

#Implied Probability OVER 2.5 goals for PINACLE closing odds
data['Implied Probability >2.5 pinacle closing']=1/data['PC>2.5']*100

#Implied Probability UNDER 2.5 goals for 365 closing odds
data['Implied Probability <2.5 365 closing']=1/data['B365C<2.5']*100

#Implied Probability OVER 2.5 goals for 365 closing odds
data['Implied Probability >2.5 365 closing']=1/data['B365C>2.5']*100

### Binning the implied probabilities

#### Binning IP Opening

In [23]:
bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]

#Binning UNDER 2.5 Average Market opening odds
data['binned <2.5 avg'] = pd.cut(data['Implied Probability <2.5 avg'], bins)

#Binning Over 2.5 Average Market opening odds
data['binned >2.5 avg'] = pd.cut(data['Implied Probability >2.5 avg'], bins)

#Binned UNDER 2.5 Pinnacle opening odds
data['binned <2.5 pinacle'] = pd.cut(data['Implied Probability <2.5 pinacle'], bins)

#Binned OVER 2.5 Pinnacle
data['binned >2.5 pinacle'] = pd.cut(data['Implied Probability >2.5 pinacle'], bins)

#Binned UNDER 2.5 bet365 OPENING odds
data['binned <2.5 365'] = pd.cut(data['Implied Probability <2.5 365'], bins)

#Binned OVER 2.5 bet365 OPENING odds
data['binned >2.5 365'] = pd.cut(data['Implied Probability >2.5 365'], bins)

#### Binning IP Closing

In [24]:
bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]

#Binning UNDER 2.5 Average Market closing odds
data['binned <2.5 avg closing'] = pd.cut(data['Implied Probability <2.5 avg closing'], bins)

#Binning OVER 2.5 Average Market closing odds
data['binned >2.5 avg closing'] = pd.cut(data['Implied Probability >2.5 avg closing'], bins)

#Binned UNDER 2.5 Pinnacle closing odds
data['binned <2.5 pinacle closing'] = pd.cut(data['Implied Probability <2.5 pinacle closing'], bins)

#Binned OVER 2.5 Pinnacle CLOSING odds
data['binned >2.5 pinacle closing'] = pd.cut(data['Implied Probability >2.5 pinacle closing'], bins)

#Binned UNDER 2.5 bet365 CLOSING odds
data['binned <2.5 365 closing'] = pd.cut(data['Implied Probability <2.5 365 closing'], bins)

#Binned OVER 2.5 bet365 CLOSING odds
data['binned >2.5 365 closing'] = pd.cut(data['Implied Probability >2.5 365 closing'], bins)

### Binning the odds

#### Binning Odds Opening

In [25]:
bins = [1, 1.5, 2, 3, 99999]

#Binning UNDER 2.5 Average Market opening odds
data['binned odds <2.5 avg'] = pd.cut(data['Avg<2.5'], bins)

#Binning Over 2.5 Average Market opening odds
data['binned odds >2.5 avg'] = pd.cut(data['Avg>2.5'], bins)

#Binned UNDER 2.5 Pinnacle opening odds
data['binned odds <2.5 pinacle'] = pd.cut(data['P<2.5'], bins)

#Binned OVER 2.5 Pinnacle
data['binned odds >2.5 pinacle'] = pd.cut(data['P>2.5'], bins)

#Binned UNDER 2.5 bet365 OPENING odds
data['binned odds <2.5 365'] = pd.cut(data['B365<2.5'], bins)

#Binned OVER 2.5 bet365 OPENING odds
data['binned odds >2.5 365'] = pd.cut(data['B365>2.5'], bins)

#### Binning Odds Closing

In [26]:
bins = [1, 1.5, 2, 3, 99999]

#Binning UNDER 2.5 Average Market opening odds
data['binned odds <2.5 avg closing'] = pd.cut(data['AvgC<2.5'], bins)

#Binning Over 2.5 Average Market opening odds
data['binned odds >2.5 avg closing'] = pd.cut(data['AvgC>2.5'], bins)

#Binned UNDER 2.5 Pinnacle opening odds
data['binned odds <2.5 pinacle closing'] = pd.cut(data['PC<2.5'], bins)

#Binned OVER 2.5 Pinnacle
data['binned odds >2.5 pinacle closing'] = pd.cut(data['PC>2.5'], bins)

#Binned UNDER 2.5 bet365 OPENING odds
data['binned odds <2.5 365 closing'] = pd.cut(data['B365C<2.5'], bins)

#Binned OVER 2.5 bet365 OPENING odds
data['binned odds >2.5 365 closing'] = pd.cut(data['B365C>2.5'], bins)

In [27]:
data['binned odds <2.5 pinacle closing'].isnull().sum()

0

### Other Features from D3

In [28]:
data['Pin_pays_better_under_boolean'] = data['PC<2.5'] > data['AvgC<2.5']

In [29]:
data['Pin_pays_better_under_difference'] = data['PC<2.5'] / data['AvgC<2.5']

In [30]:
data['%vig_p'] = (1 - (1 / (1/data['PC>2.5'] + 1/data['PC<2.5'])))*100

In [31]:
data['%vig_p_bool'] = data['%vig_p']>3.3

In [32]:
data['%vig_avg'] = (1 - (1 / (1/data['AvgC>2.5'] + 1/data['AvgC<2.5'])))*100

In [33]:
data['PC<2.5_P_boolean'] = data['PC<2.5'] < data['P<2.5']

In [34]:
data['PC<2.5_P_relative_diff'] = data['PC<2.5'] / data['P<2.5']

In [35]:
data['MaxC>2.5_AvgC_relative_diff'] = data['MaxC>2.5']/data['AvgC>2.5']
data['Market_consensus'] = data['MaxC>2.5_AvgC_relative_diff']<1.05

In [36]:
## Adding the Year Feature
data_date = data['Date']
data_time = data['Time']

data_date_2 = pd.to_datetime(data_date, dayfirst = True)
data_time_2 = pd.to_datetime(data_time, dayfirst = True)

data['month'] = pd.DatetimeIndex(data_date_2).month
data['month_after_July'] = data['month']>=7

data['year'] = pd.DatetimeIndex(data_date_2).year
data['year_2021_2022'] = data['year']>=2021

data['hour'] = pd.DatetimeIndex(data_time_2).hour
data['game_starts_after_4pm']=data['hour']>=16

In [37]:
data['game_starts_after_4pm'].sum()

5910

# [KILL] Final Dataset [KILL]

### Opening Markets

In [38]:
#Average Market UNDER opening odds
fdf_under_avg = data[['Implied Probability <2.5 avg','under_2.5_goals','binned <2.5 avg','payout_avg_under_2.5']]

#Average Market OVER opening odds
fdf_over_avg = data[['Implied Probability >2.5 avg','over_2.5_goals','binned >2.5 avg','payout_avg_over_2.5']]

#pinnacle UNDER opening odds
fdf_under_pinacle = data[['Implied Probability <2.5 pinacle','under_2.5_goals','binned <2.5 pinacle','payout_under_2.5_pinacle']]

#pinnacle OVER opening odds
fdf_over_pinacle = data[['Implied Probability >2.5 pinacle','over_2.5_goals','binned >2.5 pinacle','payout_over_2.5_pinacle']]

#bet365 UNDER opening odds
fdf_under_365 = data[['Implied Probability <2.5 365','under_2.5_goals','binned <2.5 365','payout_under_2.5_365']]

#bet365 OVER opening odds
fdf_over_365 = data[['Implied Probability >2.5 365','over_2.5_goals','binned >2.5 365','payout_over_2.5_365']]

### Closing Markets

In [39]:
#Average Market UNDER closing odds
fdf_under_avg_closing = data[['Implied Probability <2.5 avg closing','under_2.5_goals','binned <2.5 avg closing','payout_avg_under_closing_2.5']]

#Average Market OVER opening odds
fdf_over_avg_closing = data[['Implied Probability >2.5 avg closing','over_2.5_goals','binned >2.5 avg closing','payout_avg_over_closing_2.5']]

#pinnacle UNDER closing odds
fdf_under_pinacle_closing = data[['Implied Probability <2.5 pinacle closing','under_2.5_goals','binned <2.5 pinacle closing','payout_under_2.5_pinacle_closing']]

#pinnacle OVER closing odds
fdf_over_pinacle_closing = data[['Implied Probability >2.5 pinacle closing','over_2.5_goals','binned >2.5 pinacle closing','payout_over_2.5_pinacle_closing']]

#bet365 UNDER closing odds
fdf_under_365_closing = data[['Implied Probability <2.5 365 closing','under_2.5_goals','binned <2.5 365 closing','payout_under_2.5_365_closing']]

#bet365 OVER closing odds
fdf_over_365_closing = data[['Implied Probability >2.5 365 closing','over_2.5_goals','binned >2.5 365 closing','payout_over_2.5_365_closing']]

# [KILL] Results per implied prob Average Market [KILL]

### Under

#### Opening Odds

In [40]:
fdf_under_avg_agg = fdf_under_avg.groupby('binned <2.5 avg')['payout_avg_under_2.5'].agg(['count', 'mean', 'median'])

fdf_under_pinacle_agg = fdf_under_pinacle.groupby('binned <2.5 pinacle')['payout_under_2.5_pinacle'].agg(['count', 'mean', 'median'])

fdf_under_bet365_agg = fdf_under_365.groupby('binned <2.5 365')['payout_under_2.5_365'].agg(['count', 'mean', 'median'])

In [41]:
fdf_under_avg_agg.rename(columns={"count": "count_avg", "mean": "mean_avg", "median": "median_avg"}, inplace=True)
fdf_under_avg_agg.index.names = ['bin<2.5']

fdf_under_pinacle_agg.rename(columns={"count": "count_p", "mean": "mean_p", "median": "median_p"}, inplace=True)
fdf_under_pinacle_agg.index.names = ['bin<2.5']

fdf_under_bet365_agg.rename(columns={"count": "count_b365", "mean": "mean_b365", "median": "median_b365"}, inplace=True)
fdf_under_bet365_agg.index.names = ['bin<2.5']

opening_test_1 = fdf_under_avg_agg.merge(fdf_under_pinacle_agg, on='bin<2.5')
opening_test_2 = opening_test_1.merge(fdf_under_bet365_agg, on='bin<2.5')
opening_test_2

,count_avg,mean_avg,median_avg,count_p,mean_p,median_p,count_b365,mean_b365,median_b365
bin<2.5,,,,,,,,,
"(0, 5]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(5, 10]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(10, 15]",1,0.000000,0.00,2,0.000000,0.000,1,0.000000,0.000
"(15, 20]",4,0.000000,0.00,11,0.460000,0.000,10,0.000000,0.000
"(20, 25]",52,0.583077,0.00,58,0.602241,0.000,73,0.639589,0.000
"(25, 30]",126,0.695079,0.00,147,0.858435,0.000,157,0.862739,0.000
"(30, 35]",315,1.092698,0.00,392,1.039541,0.000,293,1.162116,0.000
"(35, 40]",763,0.982320,0.00,979,1.018253,0.000,842,0.966271,0.000
"(40, 45]",1520,0.895612,0.00,1570,0.938140,0.000,1101,0.896131,0.000


#### Closing Odds

In [42]:
fdf_under_avg_closing_gb_agg = fdf_under_avg_closing.groupby('binned <2.5 avg closing')['payout_avg_under_closing_2.5'].agg(['count', 'mean', 'median', mode])

fdf_under_pinacle_closing_gb_agg = fdf_under_pinacle_closing.groupby('binned <2.5 pinacle closing')['payout_under_2.5_pinacle_closing'].agg(['count', 'mean', 'median', mode])

fdf_under_bet365_closing_gb_agg = fdf_under_365_closing.groupby('binned <2.5 365 closing')['payout_under_2.5_365_closing'].agg(['count', 'mean', 'median', mode])

In [43]:
fdf_under_avg_closing_gb_agg.rename(columns={"count": "count_avg", "mean": "mean_avg", "median": "median_avg", "mode": "mode_avg"}, inplace=True)
fdf_under_avg_closing_gb_agg.index.names = ['bin<2.5']

fdf_under_pinacle_closing_gb_agg.rename(columns={"count": "count_p", "mean": "mean_p", "median": "median_p", "mode": "mode_p"}, inplace=True)
fdf_under_pinacle_closing_gb_agg.index.names = ['bin<2.5']

fdf_under_bet365_closing_gb_agg.rename(columns={"count": "count_b365", "mean": "mean_b365", "median": "median_b365", "mode": "mode_b365"}, inplace=True)
fdf_under_bet365_closing_gb_agg.index.names = ['bin<2.5']

closing_test_1 = fdf_under_avg_closing_gb_agg.merge(fdf_under_pinacle_closing_gb_agg, on='bin<2.5')
closing_test_2 = closing_test_1.merge(fdf_under_bet365_closing_gb_agg, on='bin<2.5')
closing_test_2['bins'] = closing_test_2.index
closing_test_2

,count_avg,mean_avg,median_avg,mode_avg,count_p,mean_p,median_p,mode_p,count_b365,mean_b365,median_b365,mode_b365,bins
bin<2.5,,,,,,,,,,,,,
"(0, 5]",0,NaN,NaN,"([], [])",0,NaN,NaN,"([], [])",0,NaN,NaN,"([], [])","(0, 5]"
"(5, 10]",0,NaN,NaN,"([], [])",0,NaN,NaN,"([], [])",0,NaN,NaN,"([], [])","(5, 10]"
"(10, 15]",1,0.000000,0.000,"([0.0], [1])",3,0.000000,0.000,"([0.0], [3])",1,0.000000,0.00,"([0.0], [1])","(10, 15]"
"(15, 20]",18,0.566667,0.000,"([0.0], [16])",28,0.581786,0.000,"([0.0], [25])",25,0.600000,0.00,"([0.0], [22])","(15, 20]"
"(20, 25]",54,0.739074,0.000,"([0.0], [45])",64,0.632031,0.000,"([0.0], [55])",99,0.636566,0.00,"([0.0], [84])","(20, 25]"
"(25, 30]",177,0.761977,0.000,"([0.0], [139])",218,0.915642,0.000,"([0.0], [162])",216,0.942824,0.00,"([0.0], [158])","(25, 30]"
"(30, 35]",370,1.001108,0.000,"([0.0], [249])",468,1.066944,0.000,"([0.0], [304])",365,1.096575,0.00,"([0.0], [233])","(30, 35]"
"(35, 40]",834,0.966859,0.000,"([0.0], [529])",1016,0.949006,0.000,"([0.0], [651])",850,0.916800,0.00,"([0.0], [550])","(35, 40]"
"(40, 45]",1473,0.912390,0.000,"([0.0], [899])",1505,0.969382,0.000,"([0.0], [883])",1139,0.923389,0.00,"([0.0], [688])","(40, 45]"


### Over

#### Opening Odds

In [44]:
fdf_over_avg_agg = fdf_over_avg.groupby('binned >2.5 avg')['payout_avg_over_2.5'].agg(['count', 'mean', 'median'])

fdf_over_pinacle_agg = fdf_over_pinacle.groupby('binned >2.5 pinacle')['payout_over_2.5_pinacle'].agg(['count', 'mean', 'median'])

fdf_over_bet365_agg = fdf_over_365.groupby('binned >2.5 365')['payout_over_2.5_365'].agg(['count', 'mean', 'median'])

In [45]:
fdf_over_avg_agg.rename(columns={"count": "count_avg", "mean": "mean_avg", "median": "median_avg"}, inplace=True)
fdf_over_avg_agg.index.names = ['bin>2.5']

fdf_over_pinacle_agg.rename(columns={"count": "count_p", "mean": "mean_p", "median": "median_p"}, inplace=True)
fdf_over_pinacle_agg.index.names = ['bin>2.5']

fdf_over_bet365_agg.rename(columns={"count": "count_b365", "mean": "mean_b365", "median": "median_b365"}, inplace=True)
fdf_over_bet365_agg.index.names = ['bin>2.5']

over_opening_test_1 = fdf_over_avg_agg.merge(fdf_over_pinacle_agg, on='bin>2.5')
over_opening_test_2 = over_opening_test_1.merge(fdf_over_bet365_agg, on='bin>2.5')
over_opening_test_2

,count_avg,mean_avg,median_avg,count_p,mean_p,median_p,count_b365,mean_b365,median_b365
bin>2.5,,,,,,,,,
"(0, 5]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(5, 10]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(10, 15]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(15, 20]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(20, 25]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(25, 30]",1,0.000000,0.00,2,0.000000,0.000,2,0.000000,0.00
"(30, 35]",36,0.823333,0.00,74,0.894324,0.000,56,0.935714,0.00
"(35, 40]",345,0.928493,0.00,484,1.029483,0.000,510,1.009255,0.00
"(40, 45]",1142,0.899641,0.00,1269,0.896895,0.000,882,0.855420,0.00


#### Closing Odds

In [46]:
fdf_over_avg_closing_agg = fdf_over_avg_closing.groupby('binned >2.5 avg closing')['payout_avg_over_closing_2.5'].agg(['count', 'mean', 'median'])

fdf_over_pinacle_closing_agg = fdf_over_pinacle_closing.groupby('binned >2.5 pinacle closing')['payout_over_2.5_pinacle_closing'].agg(['count', 'mean', 'median'])

fdf_over_bet365_closing_agg = fdf_over_365_closing.groupby('binned >2.5 365 closing')['payout_over_2.5_365_closing'].agg(['count', 'mean', 'median'])

In [47]:
fdf_over_avg_closing_agg.rename(columns={"count": "count_avg", "mean": "mean_avg", "median": "median_avg"}, inplace=True)
fdf_over_avg_closing_agg.index.names = ['bin>2.5']

fdf_over_pinacle_closing_agg.rename(columns={"count": "count_p", "mean": "mean_p", "median": "median_p"}, inplace=True)
fdf_over_pinacle_closing_agg.index.names = ['bin>2.5']

fdf_over_bet365_closing_agg.rename(columns={"count": "count_b365", "mean": "mean_b365", "median": "median_b365"}, inplace=True)
fdf_over_bet365_closing_agg.index.names = ['bin>2.5']

over_closing_test_1 = fdf_over_avg_closing_agg.merge(fdf_over_pinacle_closing_agg, on='bin>2.5')
over_closing_test_2 = over_closing_test_1.merge(fdf_over_bet365_closing_agg, on='bin>2.5')
over_closing_test_2

,count_avg,mean_avg,median_avg,count_p,mean_p,median_p,count_b365,mean_b365,median_b365
bin>2.5,,,,,,,,,
"(0, 5]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(5, 10]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(10, 15]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(15, 20]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(20, 25]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(25, 30]",2,0.000000,0.00,10,0.689000,0.000,21,0.819048,0.00
"(30, 35]",102,0.881275,0.00,190,0.898684,0.000,158,0.939557,0.00
"(35, 40]",493,0.951643,0.00,660,1.017318,0.000,624,0.978462,0.00
"(40, 45]",1111,0.916238,0.00,1217,0.927707,0.000,868,0.905991,0.00


# Starting Model

In [48]:
#Pedro's pet! Don't touch it!
"""odd = 2.15
iproba = 1/odd*100
for x in range(0, 20):
    if iproba in closing_test_2.iloc[6]['bins']"""

"odd = 2.15\niproba = 1/odd*100\nfor x in range(0, 20):\n    if iproba in closing_test_2.iloc[6]['bins']"

In [49]:
data_linear_booleans = data.copy()

In [50]:
data_linear_booleans_lean_P_under = data_linear_booleans[['country','month_after_July','year_2021_2022','game_starts_after_4pm','binned odds <2.5 pinacle closing','Pin_pays_better_under_boolean','Market_consensus','%vig_p_bool','PC<2.5_P_boolean', 'payout_under_2.5_pinacle_closing','0, 20','20, 40','40, 60','60, 80','80, 100']]

In [51]:
# data_model_over_under = data_model_over_under[["Div",
# "Date", "Time", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "nb_goals", "Avg>2.5", "Avg<2.5", "B365>2.5", "B365<2.5", "P>2.5", 
# "P<2.5", "AvgC>2.5", "AvgC<2.5", "B365C>2.5", "B365C<2.5", "PC>2.5", "PC<2.5", "over_2.5_goals", "under_2.5_goals", 
# "payout_avg_under_2.5", "payout_avg_over_2.5", "payout_under_2.5_pinacle", "payout_over_2.5_pinacle", "payout_under_2.5_365", 
# "payout_over_2.5_365", "payout_avg_under_closing_2.5", "payout_avg_over_closing_2.5", "payout_under_2.5_pinacle_closing", 
# "payout_over_2.5_pinacle_closing", "payout_under_2.5_365_closing", "payout_over_2.5_365_closing", "Implied Probability <2.5 avg", 
# "Implied Probability >2.5 avg", "Implied Probability <2.5 pinacle", "Implied Probability >2.5 pinacle", "Implied Probability <2.5 365", 
# "Implied Probability >2.5 365", "Implied Probability <2.5 avg closing", "Implied Probability >2.5 avg closing", "Implied Probability <2.5 pinacle closing", 
# "Implied Probability >2.5 pinacle closing", "Implied Probability <2.5 365 closing", "Implied Probability >2.5 365 closing", 
# "binned <2.5 avg", "binned >2.5 avg", "binned <2.5 pinacle", "binned >2.5 pinacle", "binned <2.5 365", "binned >2.5 365", 
# "binned <2.5 avg closing", "binned >2.5 avg closing", "binned <2.5 pinacle closing", "binned >2.5 pinacle closing", 
# "binned <2.5 365 closing", "binned >2.5 365 closing"]]

In [52]:
# data_model_over_under.dropna(inplace=True)

In [53]:
# data_model_over_under

In [54]:
# OneHotEncoders for Bin Odds

In [55]:
ohe = OneHotEncoder(sparse=False) 
ohe.fit(data_linear_booleans_lean_P_under[['binned odds <2.5 pinacle closing']])
bins_encoded = ohe.transform(data_linear_booleans_lean_P_under[['binned odds <2.5 pinacle closing']])
data_linear_booleans_lean_P_under["1.0_to_1.5"], data_linear_booleans_lean_P_under["1.5_to_2.0"], data_linear_booleans_lean_P_under["2.0_to_3.0"], data_linear_booleans_lean_P_under["3.0_to_99999.0"] = bins_encoded.T
#data_linear_booleans_lean_P_under.drop(columns='binned <2.5 pinacle', inplace=True)

/tmp/ipykernel_2640/1468458170.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_linear_booleans_lean_P_under["1.0_to_1.5"], data_linear_booleans_lean_P_under["1.5_to_2.0"], data_linear_booleans_lean_P_under["2.0_to_3.0"], data_linear_booleans_lean_P_under["3.0_to_99999.0"] = bins_encoded.T
/tmp/ipykernel_2640/1468458170.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_linear_booleans_lean_P_under["1.0_to_1.5"], data_linear_booleans_lean_P_under["1.5_to_2.0"], data_linear_booleans_lean_P_unde

In [56]:
ohe = OneHotEncoder(sparse=False) 
ohe.fit(data_linear_booleans_lean_P_under[['country']])
bins_encoded = ohe.transform(data_linear_booleans_lean_P_under[['country']])
data_linear_booleans_lean_P_under["country_1"], data_linear_booleans_lean_P_under["country_2"], data_linear_booleans_lean_P_under["country_3"], data_linear_booleans_lean_P_under["country_4"], data_linear_booleans_lean_P_under["country_5"],data_linear_booleans_lean_P_under["country_6"], data_linear_booleans_lean_P_under["country_7"], data_linear_booleans_lean_P_under["country_8"], data_linear_booleans_lean_P_under["country_9"], data_linear_booleans_lean_P_under["country_10"], data_linear_booleans_lean_P_under["country_11"] = bins_encoded.T
#data_linear_booleans_lean_P_under["country_1"], data_linear_booleans_lean_P_under["country_2"], data_linear_booleans_lean_P_under["country_3"], data_linear_booleans_lean_P_under["country_4"], data_linear_booleans_lean_P_under["country_5"], data_linear_booleans_lean_P_under["country_6"]= bins_encoded.T
#data_linear_booleans_lean_P_under.drop(columns='binned <2.5 pinacle', inplace=True)

/tmp/ipykernel_2640/925268853.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_linear_booleans_lean_P_under["country_1"], data_linear_booleans_lean_P_under["country_2"], data_linear_booleans_lean_P_under["country_3"], data_linear_booleans_lean_P_under["country_4"], data_linear_booleans_lean_P_under["country_5"],data_linear_booleans_lean_P_under["country_6"], data_linear_booleans_lean_P_under["country_7"], data_linear_booleans_lean_P_under["country_8"], data_linear_booleans_lean_P_under["country_9"], data_linear_booleans_lean_P_under["country_10"], data_linear_booleans_lean_P_under["country_11"] = bins_encoded.T
/tmp/ipykernel_2640/925268853.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [57]:
#extra
data_linear_booleans_lean_P_under = data_linear_booleans_lean_P_under.drop(['country'],axis=1)

In [58]:
#extra
data_linear_booleans_lean_P_under = data_linear_booleans_lean_P_under.drop(['binned odds <2.5 pinacle closing'],axis=1)

In [59]:
data_linear_booleans_lean_P_under.sum()

month_after_July                    5128.00
year_2021_2022                      5465.00
game_starts_after_4pm               5910.00
Pin_pays_better_under_boolean       8748.00
Market_consensus                    5737.00
%vig_p_bool                         3574.00
PC<2.5_P_boolean                    4688.00
payout_under_2.5_pinacle_closing    9618.42
0, 20                                309.00
20, 40                              2685.00
40, 60                              4891.00
60, 80                              1696.00
80, 100                              312.00
1.0_to_1.5                           420.00
1.5_to_2.0                          4365.00
2.0_to_3.0                          4528.00
3.0_to_99999.0                       580.00
country_1                            786.00
country_2                            691.00
country_3                           1048.00
country_4                            805.00
country_5                           1077.00
country_6                       

In [60]:
for col_name in data_linear_booleans_lean_P_under.columns: 
    print(col_name)

month_after_July
year_2021_2022
game_starts_after_4pm
Pin_pays_better_under_boolean
Market_consensus
%vig_p_bool
PC<2.5_P_boolean
payout_under_2.5_pinacle_closing
0, 20
20, 40
40, 60
60, 80
80, 100
1.0_to_1.5
1.5_to_2.0
2.0_to_3.0
3.0_to_99999.0
country_1
country_2
country_3
country_4
country_5
country_6
country_7
country_8
country_9
country_10
country_11


In [61]:
# test_model = data_model_over_under.drop(columns=['FTHG', 'FTAG', 'nb_goals', 'over_2.5_goals', 'under_2.5_goals', 
#                                                 'Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'binned <2.5 avg', 
#                                                 'binned >2.5 avg', 'binned >2.5 pinacle', 'binned <2.5 365', 
#                                                  'binned >2.5 365', 'binned <2.5 avg closing', 'binned >2.5 avg closing', 
#                                                 'binned <2.5 pinacle closing', 'binned >2.5 pinacle closing', 'binned <2.5 365 closing', 
#                                                 'binned >2.5 365 closing', 
#                                                 'B365>2.5', 'B365<2.5', 'B365C>2.5', 'B365C<2.5', 'payout_under_2.5_365', 'payout_over_2.5_365', 
#                                                 'payout_under_2.5_365_closing', 'payout_over_2.5_365_closing', 
#                                                 'Implied Probability <2.5 365', 'Implied Probability >2.5 365', 
#                                                 'Implied Probability <2.5 365 closing', 'Implied Probability >2.5 365 closing', 
#                                                 'payout_avg_under_2.5', 'payout_avg_over_2.5', 'payout_under_2.5_pinacle', 
#                                                 'payout_avg_over_closing_2.5', 'payout_over_2.5_pinacle_closing', 
#                                                 'Implied Probability <2.5 avg', 'Implied Probability >2.5 avg', 
#                                                 'Implied Probability <2.5 pinacle', 'Implied Probability >2.5 pinacle', 
#                                                 'Avg>2.5', 'P>2.5', 'AvgC>2.5', 'PC>2.5', 'payout_over_2.5_pinacle', 
#                                                 'Implied Probability >2.5 avg closing', 'Implied Probability >2.5 pinacle closing', 
#                                                 'payout_avg_under_closing_2.5', 'Avg<2.5', 'P<2.5', 'AvgC<2.5', 'PC<2.5', 
#                                                 'Implied Probability <2.5 avg closing', 'Implied Probability <2.5 pinacle closing'])

# test_model

In [62]:
X = data_linear_booleans_lean_P_under[['Pin_pays_better_under_boolean','Market_consensus','%vig_p_bool','PC<2.5_P_boolean', '1.0_to_1.5', '1.5_to_2.0', '2.0_to_3.0','3.0_to_99999.0']]
X_step2 = data_linear_booleans_lean_P_under[['%vig_p_bool','PC<2.5_P_boolean', '1.0_to_1.5', '1.5_to_2.0', '2.0_to_3.0','3.0_to_99999.0']]
X_step3 = data_linear_booleans_lean_P_under[['country_1','country_2','country_3','country_4','country_5','country_6','country_7','country_8','country_9','country_10','country_11','month_after_July', 'year_2021_2022', 'game_starts_after_4pm','Pin_pays_better_under_boolean','Market_consensus','%vig_p_bool','PC<2.5_P_boolean','1.0_to_1.5', '1.5_to_2.0', '2.0_to_3.0','3.0_to_99999.0','0, 20','20, 40','40, 60','60, 80','80, 100']]
#X_step4 = data_linear_booleans_lean_P_under[['country_1','country_2','country_3','country_4','country_5','country_6','country_7','country_8','country_9','country_10','country_11']]
y = data_linear_booleans_lean_P_under['payout_under_2.5_pinacle_closing']-1

X_step_joao = data_linear_booleans_lean_P_under[['month_after_July', 'year_2021_2022', 'game_starts_after_4pm','Pin_pays_better_under_boolean','Market_consensus','%vig_p_bool','PC<2.5_P_boolean','1.0_to_1.5', '1.5_to_2.0', '2.0_to_3.0','3.0_to_99999.0','0, 20','20, 40','40, 60','60, 80','80, 100']]
X_step_f= data_linear_booleans_lean_P_under[['0, 20','20, 40','40, 60','60, 80','80, 100']]


In [63]:
#X_step_f= data_linear_booleans_lean_P_under[['0, 20','20, 40','40, 60','60, 80','80, 100']]


In [64]:
#X_step3['great_countries']=X_step3['country_3']+X_step3['country_7']+X_step3['country_8']+X_step3['country_9']+X_step3['country_10']

In [65]:
#X_step3 = X_step3[X_step3['great_countries']==1]

In [66]:
#X_step3 = X_step3[['country_3','country_7','country_8','country_9','country_10','month_after_July', 'year_2021_2022', 'game_starts_after_4pm','Pin_pays_better_under_boolean','Market_consensus','%vig_p_bool','1.0_to_1.5', '1.5_to_2.0', '2.0_to_3.0','3.0_to_99999.0','0, 20','20, 40','40, 60','60, 80','80, 100']]

In [67]:
#X_step4 = data_linear_booleans_lean_P_under[['country_1','country_2','country_3','country_4','country_5','country_6','country_7','country_8','country_9','country_10','country_11']]
#y = data_linear_booleans_lean_P_under['payout_under_2.5_pinacle_closing']-

In [75]:
import statsmodels.api as sm

In [76]:
X = data_linear_booleans_lean_P_under[['country_1','country_2','country_3','country_4','country_5','country_6','country_7','country_8','country_9','country_10','country_11','month_after_July', 'year_2021_2022', 'game_starts_after_4pm','Pin_pays_better_under_boolean','Market_consensus','%vig_p_bool','PC<2.5_P_boolean','1.0_to_1.5', '1.5_to_2.0', '2.0_to_3.0','3.0_to_99999.0','0, 20','20, 40','40, 60','60, 80','80, 100']]

In [77]:
X1 = data_linear_booleans_lean_P_under[['country_1','country_2','country_3','country_4','country_5','country_6','country_7','country_8','country_9','country_10','country_11','month_after_July', 'year_2021_2022', 'game_starts_after_4pm','Pin_pays_better_under_boolean','Market_consensus','%vig_p_bool','PC<2.5_P_boolean','1.0_to_1.5', '1.5_to_2.0', '2.0_to_3.0','3.0_to_99999.0','0, 20','20, 40','40, 60','60, 80','80, 100']]

In [78]:
from sklearn.model_selection import train_test_split

# Ready X and y
X2 = X1
y = data_linear_booleans_lean_P_under['payout_under_2.5_pinacle_closing']-1

# Split into Train/Test
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.3)

In [79]:
#Linear Regression
results_linear_regression = sm.OLS(y_train,X_train.astype(float)).fit()
y_pred_lr = results_linear_regression.predict(X_test.astype(float))
#results.summary()

In [80]:
#XGBoost
import xgboost as xgb
import sklearn
m = xgb.XGBRegressor()
X_test.rename(columns = {'PC<2.5_P_boolean':'PC_under_2.5_P_boolean'}, inplace = True)
X_train.rename(columns = {'PC<2.5_P_boolean':'PC_under_2.5_P_boolean'}, inplace = True)
m.fit(X_train,y_train) 
y_pred_xgb = m.predict(X_test)
y_pred_xgb = pd.DataFrame(y_pred_xgb)
r2train = sklearn.metrics.r2_score(y_test, y_pred_xgb)

In [81]:
#Bins cutoffs
def ifnull(var, val):
    if var > 0:
        return var
    return val

#Linear Regression
y_pred_lr_under_0_median = y_pred_lr[y_pred_lr<0].median()
y_pred_lr_under_0_min = y_pred_lr[y_pred_lr<0].min()
y_pred_lr_over_0_median = ifnull(y_pred_lr[y_pred_lr>0].median(),0.05)

#XGB
y_pred_xgb_under_0_median = y_pred_xgb[y_pred_xgb<0].median()
y_pred_xgb_under_0_min = y_pred_xgb[y_pred_xgb<0].min()
y_pred_xgb_over_0_median = ifnull(y_pred_xgb[0][y_pred_xgb[0]>0].median(),0.05)

In [84]:
#Linear Regression
bins3_lr = [y_pred_lr_under_0_min, y_pred_lr_under_0_median+0.0000001, 0, y_pred_lr_over_0_median, 1]
y_lr_df = pd.DataFrame(y_test)
y_pred_lr_df = pd.DataFrame(y_pred_lr)
y_pred_lr_df["binned_pred"] = pd.cut(y_pred_lr_df[0], bins3_lr)
ind = np.arange(0, len(y_pred_lr_df))
ind = ind.tolist()
y_lr_df['ind'] = ind
y_pred_lr_df['ind'] = ind
y_final_lr = y_lr_df.merge(y_pred_lr_df, on="ind")#, on = "axis")#, how = "inner")
y_final_lr['bet_opp']=y_final_lr[0]>0

In [85]:
#XGB
bins3_xgb = [y_pred_xgb_under_0_min[0], y_pred_xgb_under_0_median[0]+0.0000001, 0, y_pred_xgb_over_0_median, 1] #int(y_pred_xgb_over_0_median[0])
y_xgb_df = pd.DataFrame(y_test)
y_pred_xgb_df = pd.DataFrame(y_pred_xgb)
y_pred_xgb_df["binned_pred"] = pd.cut(y_pred_xgb_df[0], bins3_xgb)
ind = np.arange(0, len(y_pred_xgb_df))
ind = ind.tolist()
y_xgb_df['ind'] = ind
y_pred_xgb_df['ind'] = ind
y_final_xgb = y_xgb_df.merge(y_pred_xgb_df, on="ind")#, on = "axis")#, how = "inner")
y_final_xgb['bet_opp']=y_final_xgb[0]>0

In [86]:
#Linear Regression
y_final_lr.groupby('bet_opp')['payout_under_2.5_pinacle_closing'].count()
y_final_lr.groupby('bet_opp')['payout_under_2.5_pinacle_closing'].agg(["mean", "count"])

,mean,count
bet_opp,,
False,-0.023588,2163
True,-0.067453,805


In [87]:
y_final_lr.groupby('binned_pred')['payout_under_2.5_pinacle_closing'].count()
y_final_lr.groupby('binned_pred')['payout_under_2.5_pinacle_closing'].agg(["mean", "count"])

,mean,count
binned_pred,,
"(-0.224, -0.0378]",-0.038187,1081
"(-0.0378, 0.0]",-0.013238,1081
"(0.0, 0.0188]",-0.101188,404
"(0.0188, 1.0]",-0.033466,401


In [88]:
#XGB
y_final_xgb.groupby('bet_opp')['payout_under_2.5_pinacle_closing'].count()
y_final_xgb.groupby('bet_opp')['payout_under_2.5_pinacle_closing'].agg(["mean", "count"])

,mean,count
bet_opp,,
False,-0.041906,1516
True,-0.028781,1452


In [89]:
y_final_xgb.groupby('binned_pred')['payout_under_2.5_pinacle_closing'].count()
y_final_xgb.groupby('binned_pred')['payout_under_2.5_pinacle_closing'].agg(["mean", "count"])

,mean,count
binned_pred,,
"(-1.694, -0.236]",-0.064201,757
"(-0.236, 0.0]",-0.018377,758
"(0.0, 0.187]",-0.037052,726
"(0.187, 1.0]",-0.048645,664


In [92]:
X1 = data_linear_booleans_lean_P_under[['country_1','country_2','country_3','country_4','country_5','country_6','country_7','country_8','country_9','country_10','country_11','month_after_July', 'year_2021_2022', 'game_starts_after_4pm','Pin_pays_better_under_boolean','Market_consensus','%vig_p_bool','PC<2.5_P_boolean','1.0_to_1.5', '1.5_to_2.0', '2.0_to_3.0','3.0_to_99999.0','0, 20','20, 40','40, 60','60, 80','80, 100']]

In [94]:
X1.columns

Index(['country_1', 'country_2', 'country_3', 'country_4', 'country_5',
       'country_6', 'country_7', 'country_8', 'country_9', 'country_10',
       'country_11', 'month_after_July', 'year_2021_2022',
       'game_starts_after_4pm', 'Pin_pays_better_under_boolean',
       'Market_consensus', '%vig_p_bool', 'PC<2.5_P_boolean', '1.0_to_1.5',
       '1.5_to_2.0', '2.0_to_3.0', '3.0_to_99999.0', '0, 20', '20, 40',
       '40, 60', '60, 80', '80, 100'],
      dtype='object')

In [100]:
# instanciate TPOTClassifier
tpot = TPOTRegressor(generations=4, population_size=20, verbosity=2, scoring='r2', n_jobs=-1, cv=2)
# process autoML with TPOT
tpot.fit(X_train, y_train)

# print score
print(tpot.score(X_test, y_test))

                                                                                                                        
Generation 1 - Current best internal CV score: -8.764807620609272e-05
                                                                                                                        
Generation 2 - Current best internal CV score: -8.764807620609272e-05
                                                                                                                        
Generation 3 - Current best internal CV score: -8.764807620609272e-05
                                                                                                                        
Generation 4 - Current best internal CV score: -8.764807620609272e-05
                                                                                                                        
Best pipeline: ElasticNetCV(input_matrix, l1_ratio=0.75, tol=0.01)
-0.00011112097664045528


/home/rafael/.pyenv/versions/3.8.12/envs/OnThePitch/lib/python3.8/site-packages/sklearn/metrics/_scorer.py:765: FutureWarning: sklearn.metrics.SCORERS is deprecated and will be removed in v1.3. Please use sklearn.metrics.get_scorer_names to get a list of available scorers and sklearn.metrics.get_metric to get scorer.
  warnings.warn(


In [114]:
X1

,country_1,country_2,country_3,country_4,country_5,country_6,country_7,country_8,country_9,country_10,...,PC<2.5_P_boolean,1.0_to_1.5,1.5_to_2.0,2.0_to_3.0,3.0_to_99999.0,"0, 20","20, 40","40, 60","60, 80","80, 100"
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,True,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,True,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,True,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,False,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
236,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
237,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
238,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


##### STEP FORWARD FEATURE SELECTION (SFS)

In [118]:
X1.columns

Index(['country_1', 'country_2', 'country_3', 'country_4', 'country_5',
       'country_6', 'country_7', 'country_8', 'country_9', 'country_10',
       'country_11', 'month_after_July', 'year_2021_2022',
       'game_starts_after_4pm', 'Pin_pays_better_under_boolean',
       'Market_consensus', '%vig_p_bool', 'PC<2.5_P_boolean', '1.0_to_1.5',
       '1.5_to_2.0', '2.0_to_3.0', '3.0_to_99999.0', '0, 20', '20, 40',
       '40, 60', '60, 80', '80, 100'],
      dtype='object')

In [124]:
sfs = SFS(RandomForestRegressor(n_estimators=100), k_features=27, verbose=2).fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:   14.7s finished

[2022-06-03 16:09:27] Features: 1/27 -- score: -0.00022849934188911013[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  26 out of  26 | elapsed:   15.1s finished

[2022-06-03 16:09:42] Features: 2/27 -- score: 0.00021780461557732166[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   16.1s finished

[2022-06-03 16:09:58] Features: 3/27 -- score: -1.121157872860934e-05[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_job

In [126]:
pd.DataFrame.from_dict(sfs.get_metric_dict()).T

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
1,"(17,)","[-0.00013115241360694974, 2.1950973897899573e-...",-0.000228,"(PC_under_2.5_P_boolean,)",0.000748,0.000582,0.000291
2,"(7, 17)","[0.00011649983525752194, 0.0018414569547102744...",0.000218,"(country_8, PC_under_2.5_P_boolean)",0.00144,0.001121,0.00056
3,"(4, 7, 17)","[0.0011507276066304861, 0.0021792256469541638,...",-0.000011,"(country_5, country_8, PC_under_2.5_P_boolean)",0.00186,0.001447,0.000724
4,"(4, 7, 17, 22)","[0.0023367542592473978, 0.0021678560591293916,...",0.000291,"(country_5, country_8, PC_under_2.5_P_boolean,...",0.002178,0.001695,0.000847
5,"(2, 4, 7, 17, 22)","[0.0022502722885044957, 0.0008667203720437167,...",-0.000066,"(country_3, country_5, country_8, PC_under_2.5...",0.002202,0.001713,0.000857
6,"(2, 4, 7, 17, 18, 22)","[0.002339496221717474, 8.768277025383853e-05, ...",-0.000389,"(country_3, country_5, country_8, PC_under_2.5...",0.002561,0.001992,0.000996
7,"(2, 4, 7, 17, 18, 19, 22)","[0.0027183918548223662, -4.010723614800149e-05...",-0.000844,"(country_3, country_5, country_8, PC_under_2.5...",0.004081,0.003175,0.001587
8,"(2, 4, 5, 7, 17, 18, 19, 22)","[0.0019616683838774573, 0.0014718132383974014,...",-0.002268,"(country_3, country_5, country_6, country_8, P...",0.006717,0.005226,0.002613
9,"(2, 4, 5, 7, 8, 17, 18, 19, 22)","[0.000463723207825506, 0.0013923827992718962, ...",-0.003274,"(country_3, country_5, country_6, country_8, c...",0.005754,0.004477,0.002238
10,"(2, 4, 5, 6, 7, 8, 17, 18, 19, 22)","[-0.003997278522865022, 0.0022565237179204, -0...",-0.004839,"(country_3, country_5, country_6, country_7, c...",0.006759,0.005259,0.002629


##### Exaustive Feature Selection

In [ ]:
efs = EFS(RandomForestRegressor(n_estimators=100, n_jobs=-1), min_features=10, max_features=27,scoring='r2', cv=None).fit(X_train, y_train)